# For this Poject I will look into a dataset regarding Airfligh delays in the USA
## The goals are do realize some EDA with the intent of answering buisness questions

source: https://www.kaggle.com/datasets/jimschacko/airlines-dataset-to-predict-a-delay

### Let's start by loading the libraries needed to read and show our data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
delay_data = pd.read_csv('../Data/raw/Airlines.csv')

In [3]:
delay_data.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


### It seems we have an id column which is not needed and would cause problems during the modelling 

In [4]:
delay_data=delay_data.drop(columns='id')
delay_data

,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,CO,269,SFO,IAH,3,15,205,1
1,US,1558,PHX,CLT,3,15,222,1
2,AA,2400,LAX,DFW,3,20,165,1
3,AA,2466,SFO,DFW,3,20,195,1
4,AS,108,ANC,SEA,3,30,202,0
...,...,...,...,...,...,...,...,...
539378,CO,178,OGG,SNA,5,1439,326,0
539379,FL,398,SEA,ATL,5,1439,305,0
539380,FL,609,SFO,MKE,5,1439,255,0
539381,UA,78,HNL,SFO,5,1439,313,1


### Let's quickly check out simultaniously the column types and if there are any NaN values

In [5]:
delay_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Airline      539383 non-null  object
 1   Flight       539383 non-null  int64 
 2   AirportFrom  539383 non-null  object
 3   AirportTo    539383 non-null  object
 4   DayOfWeek    539383 non-null  int64 
 5   Time         539383 non-null  int64 
 6   Length       539383 non-null  int64 
 7   Delay        539383 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 32.9+ MB


In [6]:
delay_data.isna().sum()

Airline        0
Flight         0
AirportFrom    0
AirportTo      0
DayOfWeek      0
Time           0
Length         0
Delay          0
dtype: int64

### Let's now check the amount of unique values we have on our categorical columns

In [7]:
delay_data['Airline'].nunique()

18

In [8]:
delay_data['AirportFrom'].nunique()

293

In [9]:
delay_data['AirportTo'].nunique()

293

In [10]:
delay_data['AirportFrom'].value_counts()

ATL    34449
ORD    24822
DFW    22154
DEN    19843
LAX    16657
       ...  
MMH       16
SJT       15
GUM       10
ADK        9
ABR        2
Name: AirportFrom, Length: 293, dtype: int64

### It seems that our data already came incredibly clean. We might have far too many values when it comes to the airport related columns but I'm keeping them for the sake of EDA.  
### The main thing I'll do is make the column names prettier

In [11]:
import re

In [12]:
def col_name_clean(df):
    
    df1=df.copy()
    cols=[]

    for col in df1.columns:
        split_col=re.findall('[a-zA-Z][^A-Z]*', col)
        col_name=''

        for i, name in enumerate(split_col):
            if i == 0:
                col_name=name

            else:
                col_name = col_name + '_' + name
        cols.append(col_name.lower())

    df1.columns = cols
    return df1



In [13]:
cleaned_data = col_name_clean(delay_data)
cleaned_data

,airline,flight,airport_from,airport_to,day_of_week,time,length,delay
0,CO,269,SFO,IAH,3,15,205,1
1,US,1558,PHX,CLT,3,15,222,1
2,AA,2400,LAX,DFW,3,20,165,1
3,AA,2466,SFO,DFW,3,20,195,1
4,AS,108,ANC,SEA,3,30,202,0
...,...,...,...,...,...,...,...,...
539378,CO,178,OGG,SNA,5,1439,326,0
539379,FL,398,SEA,ATL,5,1439,305,0
539380,FL,609,SFO,MKE,5,1439,255,0
539381,UA,78,HNL,SFO,5,1439,313,1


### "time" and "length" aren't as descriptive as they could be, so we're changing the name to something more clear

In [14]:
cleaned_data.rename(columns={'time':'departure_time', 'length':'flight_length'}, inplace=True)
cleaned_data

,airline,flight,airport_from,airport_to,day_of_week,departure_time,flight_length,delay
0,CO,269,SFO,IAH,3,15,205,1
1,US,1558,PHX,CLT,3,15,222,1
2,AA,2400,LAX,DFW,3,20,165,1
3,AA,2466,SFO,DFW,3,20,195,1
4,AS,108,ANC,SEA,3,30,202,0
...,...,...,...,...,...,...,...,...
539378,CO,178,OGG,SNA,5,1439,326,0
539379,FL,398,SEA,ATL,5,1439,305,0
539380,FL,609,SFO,MKE,5,1439,255,0
539381,UA,78,HNL,SFO,5,1439,313,1


### Finally the departure_time column has the time of the day in minutes starting from midnight, which is fairly simple to fix

In [15]:
def min_to_h(x):
    
    y = str(x/60)
    z = y.split('.')
    h = z[0]
    min = str(x % 60)
    hour = h + ':' + min
    return hour
        


In [16]:
cleaned_data['departure_time'] = cleaned_data['departure_time'].apply(min_to_h)
cleaned_data['departure_time']

0          0:15
1          0:15
2          0:20
3          0:20
4          0:30
          ...  
539378    23:59
539379    23:59
539380    23:59
539381    23:59
539382    23:59
Name: departure_time, Length: 539383, dtype: object

### Now that it's being correctly displayed we also need to change to column type appropriately

In [17]:
cleaned_data['departure_time'] = pd.to_datetime(cleaned_data['departure_time'],format= '%H:%M' ).dt.time
cleaned_data

,airline,flight,airport_from,airport_to,day_of_week,departure_time,flight_length,delay
0,CO,269,SFO,IAH,3,00:15:00,205,1
1,US,1558,PHX,CLT,3,00:15:00,222,1
2,AA,2400,LAX,DFW,3,00:20:00,165,1
3,AA,2466,SFO,DFW,3,00:20:00,195,1
4,AS,108,ANC,SEA,3,00:30:00,202,0
...,...,...,...,...,...,...,...,...
539378,CO,178,OGG,SNA,5,23:59:00,326,0
539379,FL,398,SEA,ATL,5,23:59:00,305,0
539380,FL,609,SFO,MKE,5,23:59:00,255,0
539381,UA,78,HNL,SFO,5,23:59:00,313,1


In [18]:
cleaned_data.dtypes

airline           object
flight             int64
airport_from      object
airport_to        object
day_of_week        int64
departure_time    object
flight_length      int64
delay              int64
dtype: object

## Time to save this "cleaned" data 

In [19]:
cleaned_data.to_csv('../Data/clean/cleaned_airlines.csv', index=False)